In [1]:
import numpy as np
import pandas as pd


In [2]:
# the number of segments; when we deploy repeater
N = np.array([1,2,4,8,16,32,64,128])

# metal parameter
metal_r = 120 / 1e-6
metal_c = (0.2) * 1e-15 / 1e-6
WL_length = 256 * 1e-6

# cell parameter
cell_num = 256
cell_c = 1e-15
cell_C = cell_c * cell_num

###################################################################
### TR/Logic Gates parameters: based on P/N ratio 7:3 (2.57:1.1)###
###################################################################

# TR parameter
# gate cap
tau = 4.5 * 1e-12 #measured FO1 delay : tau(1+pinv)
TR_C = (1.5/1.1) * 1e-15 # measured
#tau = (10/3) * TR_C * TR_R
TR_R = tau / ((10/3)*TR_C)

# Logic Gates
g_inv, p_inv = 1, 2.29

g_nand2, g_nand3, g_nand6 = 1.3 , 1.6, 2.5
p_nand2, p_nand3, p_nand6 = 2.29*2, 2.29*3, 2.29*6

g_nor2, g_nor3 = 1.7, 2.4
p_nor2, p_nor3 = 2.29*2, 2.29*3

# Decoder Logic
G_nand6_inv = g_nand6 * g_inv
P_nand6_inv = p_nand6 + p_inv
G_nand3_nor2 = g_nand3 * g_nor2
P_nand3_nor2 = p_nand3 + p_nor2
B = 32 # critical path: CLK signal

In [3]:
# delay of decoder
WL_R = metal_r * WL_length
WL_C = metal_c * WL_length

C_out = (WL_C + cell_C)
C_in = 5 * 1e-15 * 32
H = C_out/C_in

G = np.array([G_nand6_inv, G_nand3_nor2])
F = np.outer(G,H)*B

F = pd.DataFrame(data = F, columns = ['value'], index=['NAND6-INV','NAND3-NOR2'])
stages = np.log(F)/np.log(4)
stages

,value
NAND6-INV,3.631517
NAND3-NOR2,3.692356


In [4]:
#with 2 stages
f_2 = F ** (1/2)
delay_decoder_stage2 = 2*f_2
delay_decoder_stage2.loc['NAND6-INV'] = delay_decoder_stage2.loc['NAND6-INV'] + P_nand6_inv
delay_decoder_stage2.loc['NAND3-NOR2'] = delay_decoder_stage2.loc['NAND3-NOR2'] + P_nand3_nor2
delay_decoder_stage2 = delay_decoder_stage2 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage2','NAND3-NOR2':'NAND3-NOR2 Stage2'}
delay_decoder_stage2.rename(index = new_row_names, inplace = True)
delay_decoder_stage2

,value
NAND6-INV Stage2,1.836769e-10
NAND3-NOR2 Stage2,1.678713e-10


In [5]:
#with 4 stages
f_4 = F ** (1/4)
delay_decoder_stage4 = 4*f_4
delay_decoder_stage4.loc['NAND6-INV'] = delay_decoder_stage4.loc['NAND6-INV'] + P_nand6_inv + 2 * p_inv
delay_decoder_stage4.loc['NAND3-NOR2'] = delay_decoder_stage4.loc['NAND3-NOR2'] + P_nand3_nor2 + 2 * p_inv
delay_decoder_stage4 = delay_decoder_stage4 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage4','NAND3-NOR2':'NAND3-NOR2 Stage4'}
delay_decoder_stage4.rename(index = new_row_names, inplace = True)
delay_decoder_stage4

,value
NAND6-INV Stage4,1.561130e-10
NAND3-NOR2 Stage4,1.368534e-10


In [6]:
#with 6 stages
f_6 = F ** (1/6)
delay_decoder_stage6 = 6*f_6
delay_decoder_stage6.loc['NAND6-INV'] = delay_decoder_stage6.loc['NAND6-INV'] + P_nand6_inv + 4 * p_inv
delay_decoder_stage6.loc['NAND3-NOR2'] = delay_decoder_stage6.loc['NAND3-NOR2'] + P_nand3_nor2 + 4 * p_inv
delay_decoder_stage6 = delay_decoder_stage6 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage6','NAND3-NOR2':'NAND3-NOR2 Stage6'}
delay_decoder_stage6.rename(index = new_row_names, inplace = True)
delay_decoder_stage6

,value
NAND6-INV Stage6,1.758381e-10
NAND3-NOR2 Stage6,1.561126e-10


In [7]:
#with 8 stages
f_8 = F ** (1/8)
delay_decoder_stage8 = 8*f_8
delay_decoder_stage8.loc['NAND6-INV'] = delay_decoder_stage8.loc['NAND3-NOR2'] + P_nand6_inv + 6 * p_inv
delay_decoder_stage8.loc['NAND3-NOR2'] = delay_decoder_stage8.loc['NAND3-NOR2'] + P_nand3_nor2 + 6 * p_inv
delay_decoder_stage8 = delay_decoder_stage8 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage8','NAND3-NOR2':'NAND3-NOR2 Stage8'}
delay_decoder_stage8.rename(index = new_row_names, inplace = True)
delay_decoder_stage8

,value
NAND6-INV Stage8,2.022272e-10
NAND3-NOR2 Stage8,1.816172e-10


In [8]:
delay_decoder = pd.concat([delay_decoder_stage2, delay_decoder_stage4, delay_decoder_stage6, delay_decoder_stage8])
delay_decoder

,value
NAND6-INV Stage2,1.836769e-10
NAND3-NOR2 Stage2,1.678713e-10
NAND6-INV Stage4,1.561130e-10
NAND3-NOR2 Stage4,1.368534e-10
NAND6-INV Stage6,1.758381e-10
NAND3-NOR2 Stage6,1.561126e-10
NAND6-INV Stage8,2.022272e-10
NAND3-NOR2 Stage8,1.816172e-10


In [9]:
# NMOS width of repeater derived from RC delay of N segments
W = ( N*(TR_R*(metal_c + 1e-15))/((10/3)*metal_r*TR_C) ) ** (1/2)
segment_delay = (TR_R/W) * ( (10*TR_C*p_inv*W/3) + cell_C/(2*N) + WL_C/(2*N) ) + (TR_R/W + WL_R/N) * ( (10*TR_C*W/3) + cell_C/(2*N) + WL_C/(2*N) )
last_segment_delay = (TR_R/W) * ( (10*TR_C*p_inv*W/3) + cell_C/(2*N) + WL_C/(2*N) ) + (TR_R/W + WL_R/N) * (cell_C/(2*N) + WL_C/(2*N))
repeater_delay = 14.8 *1e-12 #measured
delay_WL = (N-1) * segment_delay + last_segment_delay   + N * repeater_delay
delay_WL

array([5.24847685e-09, 2.83042843e-09, 1.67215352e-09, 1.20884273e-09,
       1.20577594e-09, 1.64059040e-09, 2.68957234e-09, 4.81081128e-09])

In [10]:
delay_total = delay_decoder.copy()
for i in range(7):
  delay_total = pd.concat([delay_total, delay_decoder], axis=1)
delay_total = delay_total + delay_WL
delay_total.columns = [1,2,4,8,16,32,64,128]
delay_total

,1,2,4,8,16,32,64,128
NAND6-INV Stage2,5.432154e-09,3.014105e-09,1.855830e-09,1.392520e-09,1.389453e-09,1.824267e-09,2.873249e-09,4.994488e-09
NAND3-NOR2 Stage2,5.416348e-09,2.998300e-09,1.840025e-09,1.376714e-09,1.373647e-09,1.808462e-09,2.857444e-09,4.978683e-09
NAND6-INV Stage4,5.404590e-09,2.986541e-09,1.828267e-09,1.364956e-09,1.361889e-09,1.796703e-09,2.845685e-09,4.966924e-09
NAND3-NOR2 Stage4,5.385330e-09,2.967282e-09,1.809007e-09,1.345696e-09,1.342629e-09,1.777444e-09,2.826426e-09,4.947665e-09
NAND6-INV Stage6,5.424315e-09,3.006266e-09,1.847992e-09,1.384681e-09,1.381614e-09,1.816428e-09,2.865410e-09,4.986649e-09
NAND3-NOR2 Stage6,5.404589e-09,2.986541e-09,1.828266e-09,1.364955e-09,1.361889e-09,1.796703e-09,2.845685e-09,4.966924e-09
NAND6-INV Stage8,5.450704e-09,3.032656e-09,1.874381e-09,1.411070e-09,1.408003e-09,1.842818e-09,2.891800e-09,5.013038e-09
NAND3-NOR2 Stage8,5.430094e-09,3.012046e-09,1.853771e-09,1.390460e-09,1.387393e-09,1.822208e-09,2.871190e-09,4.992428e-09


In [11]:
delay_total.describe()

,1,2,4,8,16,32,64,128
count,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00
mean,5.418516e-09,3.000467e-09,1.842192e-09,1.378881e-09,1.375815e-09,1.810629e-09,2.859611e-09,4.980850e-09
std,2.029643e-11,2.029643e-11,2.029643e-11,2.029643e-11,2.029643e-11,2.029643e-11,2.029643e-11,2.029643e-11
min,5.385330e-09,2.967282e-09,1.809007e-09,1.345696e-09,1.342629e-09,1.777444e-09,2.826426e-09,4.947665e-09
25%,5.404590e-09,2.986541e-09,1.828266e-09,1.364956e-09,1.361889e-09,1.796703e-09,2.845685e-09,4.966924e-09
50%,5.420332e-09,3.002283e-09,1.844008e-09,1.380697e-09,1.377631e-09,1.812445e-09,2.861427e-09,4.982666e-09
75%,5.430609e-09,3.012561e-09,1.854286e-09,1.390975e-09,1.387908e-09,1.822723e-09,2.871704e-09,4.992943e-09
max,5.450704e-09,3.032656e-09,1.874381e-09,1.411070e-09,1.408003e-09,1.842818e-09,2.891800e-09,5.013038e-09


In [12]:
H

1.92

In [13]:
f_4

,value
NAND6-INV,3.520447
NAND3-NOR2,3.595465


In [14]:
W

array([0.60249632, 0.85205847, 1.20499264, 1.70411693, 2.40998528,
       3.40823387, 4.81997056, 6.81646773])

In [15]:
2.40998528 * 7 / 3

5.623298986666666